# MC56F81xxx/MWCT2xxx Device HSM and Secure Boot

This notebook describes how to setup basic secure boot on MC56F81xxx/MWCT2xxx devices using the SPSDK *nxpdevhsm* and *nxpimage* applications.

Prepare a plain binary image as an input for the *nxpimage* tool. There are two sample applications in workspace with OPEN and CLOSED lifecycles.

In [10]:
%run ../init_notebook.ipynb

import os
import pprint

pp = pprint.PrettyPrinter(indent=4)

WORKSPACE = "workspace/"  # change this to path to your workspace
DATA = "data/"  # data folder that contains input images
VERBOSITY = (
    "-v"  # verbosity of commands, might be -v or -vv for debug or blank for no additional info
)

env: JUPYTER_SPSDK=1
Created `%!` as an alias for `%execute`.


## Signed image (MBI)

In the picture below there's a layout of signed image.

![Signed Image Layout](img/image_layout.png "Signed Image Layout")

### Certificate block

First step is to create certificate block and then signed image.

Certificate block configuration contains ISK certificate, self-signed or NXP signed. If the key is self-signed, signing key or signature provider must be provided. In this example we will use already prepared key-pair from the workspace directory. User should prepare their own key-pair using the nxpcrypto application.

You might use *get-template* command to obtain template configuration for cert-block.

```yaml
# =====================================  Certification Block for MC56xx template  ======================================

# ======================================================================================================================
#                                             == ISK Certificate Settings ==
# ======================================================================================================================
# -------------===== True if certificate is self signed, false for NXP Signed certificate [Optional] =====--------------
# Description: NXP Signed = false, Self signed = True
selfSigned: true
# ---------------------------------===== ISK public key [Conditionally required] =====----------------------------------
# Description: Path to ISK public key.
iskPublicKey: ec_secp256r1_sign_cert.pem
# -----------------------===== Main root Certification Private Key [Conditionally required] =====-----------------------
# Description: Path to Main root Certification Private Key.
# signPrivateKey: main_cert_prv_key.pem
# -------------------------------===== Signature Provider [Conditionally required] =====--------------------------------
# Description: Signature provider configuration in format 'type=<sp_type>;<key1>=<value1>;<key2>=<value2>".
signProvider: type=file;file_path=ecc_secp256r1_priv_key.pem
# ======================================================================================================================
#                                                  == Basic Settings ==
# ======================================================================================================================
# --------------------------------------===== cert block filename [Required] =====--------------------------------------
# Description: Generated cert block filename.
containerOutputFile: cert_block.bin

```

In [12]:
# generate template for cert block
CERT_BLOCK_TEMPLATE = WORKSPACE + "cert_block_template.yaml"
# generate template for mbi
TEMPLATES_PATH = WORKSPACE + "templates"
# choose family for the MCU
FAMILY = "mwct2xd2"

# generate template for cert block
%! nxpimage $VERBOSITY cert-block get-template -f $FAMILY -o $CERT_BLOCK_TEMPLATE --force
# generate template for MBI
%! nxpimage $VERBOSITY mbi get-templates -f $FAMILY -o $TEMPLATES_PATH --force
# just for verification that the template was generated
assert os.path.exists(os.path.join(TEMPLATES_PATH, "mwct2xd2_xip_signed.yaml"))
assert os.path.exists(CERT_BLOCK_TEMPLATE)

nxpimage -v cert-block get-template -f mwct2xd2 -o workspace/cert_block_template.yaml --force 
Creating workspace/cert_block_template.yaml template file.
nxpimage -v mbi get-templates -f mwct2xd2 -o workspace/templates --force 
Creating workspace/templates/mwct2xd2_xip_signed.yaml template file.
Creating workspace/templates/mwct2xd2_xip_plain.yaml template file.


### Generate signed image (MBI) and Cert Block

Template configuration for MBI might be created with *get-templates* command with proper device family (mc56f818xx or mwct2xd2).
The only supported outputImageAuthenticationType is *signed* and outputImageExecutionTarget is *xip*.

User must provide input binary file built with CodeWarrior that contains properly configured BCA and FCB sections. (Do not forget the lifecycle!. Lifecycle is setup in the application - flash.h)

*addCertHash* option will append certificate hash to the offset 0x4A0. This should be only used to test the image before writing certificate hash to IFR. After IFR has been written, it's no longer necessary to append hash to image.

User should also provide private key or signature provider that will be used for image signature.

> Generated MBI should be split to two parts at 0x800 and 0XC00 to exclude DUK Certificate block. This part of memory is write protected, when using the blhost write-memory. However, it could be written using the SBx.

```yaml
# =================  Master Boot Image Configuration template for mc56f818xx, Plain Signed XIP Image.  =================

# ======================================================================================================================
#                                                  == Basic Settings ==                                                 
# ======================================================================================================================
# ------------------------------------------===== MCU family [Required] =====-------------------------------------------
# Description: MCU family name.
family: mc56f818xx
# --------------------------------------===== Application target [Required] =====---------------------------------------
# Description: Definition if application is Execute in Place(XiP) or loaded to RAM during reset sequence.
# Possible options: <xip, load-to-ram>
outputImageExecutionTarget: xip
# -------------------------------===== Type of boot image authentication [Required] =====-------------------------------
# Description: Specification of final master boot image authentication.
# Possible options: <plain, crc, signed, signed-encrypted, signed-nxp>
outputImageAuthenticationType: signed
# ---------------------------------------===== Output Image name [Required] =====---------------------------------------
# Description: The path for result binary file.
masterBootOutputFile: my_mbi.bin
# ------------------------------------===== Plain application image [Required] =====------------------------------------
# Description: The input application image to by modified to Master Boot Image.
inputImageFile: my_application.bin
# ---------------------------------------===== Firmware version. [Optional] =====---------------------------------------
# Description: Value compared with Secure_FW_Version monotonic counter value stored in PFR/IFR. If value is lower than
# value in PFR/IFR, then is image rejected (rollback protection)..
firmwareVersion: 0
# ---------------------------------------===== Life cycle state [Optional] =====----------------------------------------
# Description: life cycle state model to protect code from reading from the device internal flash, which is called code
# read protection feature. There are different levels of protections in the system, so that access to the on-chip flash
# and use of ISP can be restricted. Four levels of Read Out Protection (ROP) are supported. This read out protection is
# a mechanism that allows user to enable different levels of protection in the system. It is a 8-bit field FTFA_FSEC
# associated with flash security byte (0x40c byte address). It can be programmed by customer. Check the SRM for more
# information. If not provided, value in application will be used.
# Possible options: <NOT_SET, OEM_OPEN, OEM_CLOSED_ROP1, OEM_CLOSED_ROP2, OEM_CLOSED_ROP3, OEM_CLOSED_NO_RETURN>
lifeCycle: OEM_OPEN
# ======================================================================================================================
#                                               == Certificate Block vX ==                                              
# ======================================================================================================================
# ---------------------------------===== Certificate Block binary file [Required] =====---------------------------------
# Description: Path to certificate block.
certBlock: cert_block.bin
# --------------===== True to append SHA256 hash [0:127] at the end of certificate block [Optional] =====---------------
# Description: Not mandatory if the hash is written in OTP
addCertHash: true
# ======================================================================================================================
#                                              == Image Signing Settings ==                                             
# ======================================================================================================================
# --------------------------===== Main Certificate private key [Conditionally required] =====---------------------------
# Description: Main Certificate private key used to sign certificate. It can be replaced by signProvider key.
signPrivateKey: main_prv_key.pem
# -------------------------------===== Signature Provider [Conditionally required] =====--------------------------------
# Description: Signature provider configuration in format 'type=<sp_type>;<key1>=<value1>;<key2>=<value2>".
signProvider: type=file;file_path=my_prv_key.pem
```


In [13]:
MBI_CONFIG_PATH = DATA + "mbi_xip.yaml"
CERT_BLOCK_CONFIG_PATH = DATA + "cert_block.yaml"

# # export cert block and Master Boot Image
%! nxpimage $VERBOSITY cert-block export -c $CERT_BLOCK_CONFIG_PATH -f $FAMILY
%! nxpimage $VERBOSITY mbi export -c $MBI_CONFIG_PATH


# # Split MBI into two images at offset 0x0800 and 0x0C00
%! nxpimage  utils binary-image extract -b ./workspace/mb_xip_signed.bin -a 0 -s 0x800 -o ./workspace/mb_xip_signed_0_0x800.bin
%! nxpimage  utils binary-image extract -b ./workspace/mb_xip_signed.bin -a 0xC00 -s 0 -o ./workspace/mb_xip_signed_0xC00.bin


# Generate MBI for closed lifecycle
MBI_CONFIG_PATH_CLOSED = DATA + "mbi_xip_closed.yaml"
%! nxpimage $VERBOSITY mbi export -c $MBI_CONFIG_PATH_CLOSED

# Split MBI into two images at offset 0x0800 and 0x0C00
%! nxpimage  utils binary-image extract -b ./workspace/mb_xip_signed_closed.bin -a 0 -s 0x800 -o ./workspace/mb_xip_signed_closed_0_0x800.bin
%! nxpimage  utils binary-image extract -b ./workspace/mb_xip_signed_closed.bin -a 0xC00 -s 0 -o ./workspace/mb_xip_signed_closed_0xC00.bin

nxpimage -v cert-block export -c data/cert_block.yaml -f mwct2xd2 
ISK Certificate hash [0:127]: 262aa32b6049583289a083cdd999a291

OTP script written to: workspace/otp_script.bcf
Success. (Certificate Block: workspace/cert_block.bin created.)
nxpimage -v mbi export -c data/mbi_xip.yaml 
INFO:spsdk.image.mbi.mbi_mixin:Cert block info: Certificate block version x
ISK Certificate:
ISK Certificate lite
Constraints:     1
Public Key:      ECC (EccCurve.SECP256R1) Public key: 
x(0xe21f93262a3f39277eb7d60b2f809413321ae9f4e91f63446ef6747e0bf87fd8) 
y(0x34773fb557e7673dab53bcbd9e7c80e408dcedfb5da2ff83253fd2e2ab6569de)

Certificate hash: 262aa32b6049583289a083cdd999a291
INFO:spsdk.apps.nxpimage:
+==0x0000_0000= Application Block ==========+
|          Size: 3.7 kiB; 3,776 B           |
|+==0x0000_0000= Vector Table =============+|
||               Size: 864 B               ||
|+==0x0000_035f============================+|
|+==0x0000_0360= Image Hash ===============+|
||               Size: 32 B  

### ISP Mode

Prepare device to ISP mode. Connect UART. If the field "peripheralDetection Timeout" in BCA is not set. Default 0xFF means 5 seconds waiting. So issue "nxpdevscan -p" command immediately after powering up the MCU. 

In [14]:
# nxpdevscan, list all UART devices
%! nxpdevscan -p

nxpdevscan -p 
-------- Connected NXP UART Devices --------

Port: COM89
Type: mboot device



### Programming IFR

Secure boot might be tested with the hash included in the image. If the secure boot works, we might program IFR using the generated blhost script with *blhost batch* command or using the OEM Provisioning SBx.


In [15]:
# Set COM port from the previous cell
COM_PORT = "com89"
OTP_SCRIPT = WORKSPACE + "otp_script.bcf"

# You might test the MBI with blhost write-memory, first erase memory
# %! blhost -p $COM_PORT flash-erase-region 0 0x800
# %! blhost -p $COM_PORT write-memory 0 workspace/mb_xip_signed_0_0x800.bin
# %! blhost -p $COM_PORT flash-erase-region 0xc00 0x800
# %! blhost -p $COM_PORT write-memory 0xC00 workspace/mb_xip_signed_0xC00.bin

# Uncomment this if you want to program IFR (WARNING! Irreversible operation)
# %! blhost -p $COM_PORT batch $OTP_SCRIPT

## Secure FW Update

Secure update is the process used to securely update the firmware image in the field. The firmware image is encrypted using lightweight symmetric cryptography algorithm and signed using ECDSA P-256, following the SBX firmware image format. 

Secure update guarantees authenticity and confidentiality of the new firmware image. It also ensures that the new image is up-to-date, preventing the rollback to an older image. Running firmware is in charge of receiving and verifying the new firmware image. 

The follow-up secure boot verifies the new firmware image again, making sure the Immutable RoT is still in charge of ensuring authenticity of the latest firmware. 
The secure update process includes Secure Binary (SB) container, which encrypts and encapsulates user firmware. Resultant SB file can be distributed to the OEM contract manufacturer, which ensures product manufacturing. Eventually the SB file can be used for product firmware update if it is supported by the product.

SB container supports two ways of firmware update:
·	OEM/CM Factory Initial Firmware Provisioning (OEM_PROVISIONING)
·	OEM In-field Firmware Update (OEM)
	The OEM/CM Factory Initial Firmware Provisioning can be used for initial device programming on the production line. It ensures confidentiality and integrity of programmed user image.


### Device HSM OEM Provisioning

![SBX Structure ](img/sbx.png "SBX Structure")

Create Initial OEM provisioning SBx file using the nxpdevhsm application. In this example we just erase the memory and program the application and reset the MCU.

Template can be created using the *nxpdevhsm get-template* command.

Our configuration for initial OEM provisioning might look like this:

```yaml
# Copyright 2023 NXP
#
# SPDX-License-Identifier: BSD-3-Clause
---
# ===========  Secure Binary X Configuration template for mc56f818xx.  ===========
# ----------------------------------------------------------------------------------------------------
#                                         == Basic Settings ==
# ----------------------------------------------------------------------------------------------------
firmwareVersion: 0 # [Optional], Firmware version., Version of application image firmware.
containerOutputFile: workspace/my_new.sbx # [Required], SBx filename, Generated SBx container filename.
family: mc56f818xx # [Required], MCU family, MCU family name., Possible options:['mc56f818xx']
# ----------------------------------------------------------------------------------------------------
#                                    == Secure Binary X Settings ==
# ----------------------------------------------------------------------------------------------------
image_type: OEM_PROVISIONING # [Optional], image type, SecureBinaryXType, Possible options:['NXP_PROVISIONING', 'OEM_PROVISIONING', 'OEM']
description: This is description of generated SB file. # [Optional], Description, Description up to 16 characters, longer will be truncated. Stored in SBx manifest.
# ----------------------------------------------------------------------------------------------------
#                               == Secure Binary X Commands Settings ==
# ----------------------------------------------------------------------------------------------------
commands: # [Required], SBx.1 Commands, Secure Binary X commands block, list of all possible options - Modify it according to your application
  # ----------------------------------------------------------------------------------------------------
  #          == List of possible 5 options. Option types[object,object,object,object,object] ==
  # ----------------------------------------------------------------------------------------------------
  - # [Example of possible configuration #0]
    erase: # [Required], Execute, Address is the jump-to address. No further processing of SB after jump, ROM do not expect to return.
      address: 0 # [Required], Address, Jump-to address to start execute code.
      size: 0x800

  - # [Example of possible configuration #0]
    load: # [Required], Execute, Address is the jump-to address. No further processing of SB after jump, ROM do not expect to return.
      address: 0 # [Required], Address, Jump-to address to start execute code.
      file: mb_xip_signed_0_0x800.bin

  - # [Example of possible configuration #0]
    erase: # [Required], Execute, Address is the jump-to address. No further processing of SB after jump, ROM do not expect to return.
      address: 0xC00 # [Required], Address, Jump-to address to start execute code.
      size: 0x800

  - # [Example of possible configuration #0]
    load: # [Required], Execute, Address is the jump-to address. No further processing of SB after jump, ROM do not expect to return.
      address: 0x0c00 # [Required], Address, Jump-to address to start execute code.
      file: mb_xip_signed_0xC00.bin

  - # [Example of possible configuration #4]
    reset: true # [Required], Reset

```


In [16]:
SBX_WORKSPACE = WORKSPACE + "sbx_ws/"
SBX_CONFIG = WORKSPACE + "sbx_oem.yaml"
SBX_FILE = WORKSPACE + "oem_open.sbx"
OEM_SHARE_INPUT = WORKSPACE + "OEM_SHARE_INPUT.BIN"

%! nxpdevhsm $VERBOSITY generate -f $FAMILY -c $SBX_CONFIG -p $COM_PORT -w $SBX_WORKSPACE -i $OEM_SHARE_INPUT

nxpdevhsm -v generate -f mwct2xd2 -c workspace/sbx_oem.yaml -p com89 -w workspace/sbx_ws/ -i workspace/OEM_SHARE_INPUT.BIN 
INFO:spsdk.mboot.mcuboot:Connect: identifier='uart', device=com89
 1: Initial target reset is disabled 
 2: Generating OEM master share.
INFO:spsdk.mboot.mcuboot:CMD: WriteMemory(address=0x04003000, length=16, mem_id=0)
INFO:spsdk.mboot.mcuboot:CMD: GetProperty(MaxPacketSize, index=0)
INFO:spsdk.mboot.mcuboot:CMD: Status: 0 (0x0) Success.
INFO:spsdk.mboot.mcuboot:CMD: Max Packet Size = 32
INFO:spsdk.mboot.mcuboot:CMD: Status: 0 (0x0) Success.
INFO:spsdk.mboot.mcuboot:CMD: Successfully Send 16 out of 16 Bytes
INFO:spsdk.mboot.mcuboot:CMD: [TrustProvisioning] DSC HSM CREATE SESSION
INFO:spsdk.mboot.mcuboot:CMD: Status: 0 (0x0) Success.
INFO:spsdk.mboot.mcuboot:CMD: ReadMemory(address=0x04003100, length=52, mem_id=0)
INFO:spsdk.mboot.mcuboot:CMD: Status: 0 (0x0) Success.
INFO:spsdk.mboot.mcuboot:CMD: Successfully Received 52 from 52 Bytes
 3: Creating SBx header.
 4:

In [17]:
# Receive SBx file
# SBX_FILE = WORKSPACE + "oem.sbx"
SBX_FILE = WORKSPACE + "my_new.sbx"


%! blhost -p $COM_PORT receive-sb-file $SBX_FILE

blhost -p com89 receive-sb-file workspace/my_new.sbx 
Sending SB file
Response status = 10119 (0x2787) ROM Loader: Pending Jump Command.


### OEM in-field update

To create a secure binary container that might be used for in-field software update we use "OEM" type in SBx configuration file. We also have to specify private key or signature provider that will be used for signing the container.

Our configuration might look like this:
```yaml
# Copyright 2023 NXP
#
# SPDX-License-Identifier: BSD-3-Clause
---
# ===========  Secure Binary X Configuration template for mc56f818xx.  ===========
# ----------------------------------------------------------------------------------------------------
#                                         == Basic Settings ==
# ----------------------------------------------------------------------------------------------------
firmwareVersion: 0 # [Optional], Firmware version., Version of application image firmware.
containerOutputFile: workspace/oem_open.sbx # [Required], SBx filename, Generated SBx container filename.
family: mc56f818xx # [Required], MCU family, MCU family name., Possible options:['mc56f818xx']
# ----------------------------------------------------------------------------------------------------
#                                    == Secure Binary X Settings ==
# ----------------------------------------------------------------------------------------------------
image_type: OEM # [Optional], image type, SecureBinaryXType, Possible options:['NXP_PROVISIONING', 'OEM_PROVISIONING', 'OEM']
description: This is description of generated SB file. # [Optional], Description, Description up to 16 characters, longer will be truncated. Stored in SBx manifest.
signingCertificatePrivateKeyFile: ec_pk_secp256r1_sign_cert.pem
# ----------------------------------------------------------------------------------------------------
#                               == Secure Binary X Commands Settings ==
# ----------------------------------------------------------------------------------------------------
commands: # [Required], SBx.1 Commands, Secure Binary X commands block, list of all possible options - Modify it according to your application
  # ----------------------------------------------------------------------------------------------------
  #          == List of possible 5 options. Option types[object,object,object,object,object] ==
  # ----------------------------------------------------------------------------------------------------
  - # [Example of possible configuration #0]
    erase: # [Required], Execute, Address is the jump-to address. No further processing of SB after jump, ROM do not expect to return.
      address: 0 # [Required], Address, Jump-to address to start execute code.
      size: 0x800

  - # [Example of possible configuration #0]
    load: # [Required], Execute, Address is the jump-to address. No further processing of SB after jump, ROM do not expect to return.
      address: 0 # [Required], Address, Jump-to address to start execute code.
      file: mb_xip_signed_0_0x800.bin

  - # [Example of possible configuration #0]
    erase: # [Required], Execute, Address is the jump-to address. No further processing of SB after jump, ROM do not expect to return.
      address: 0xC00 # [Required], Address, Jump-to address to start execute code.
      size: 0x800

  - # [Example of possible configuration #0]
    load: # [Required], Execute, Address is the jump-to address. No further processing of SB after jump, ROM do not expect to return.
      address: 0x0c00 # [Required], Address, Jump-to address to start execute code.
      file: mb_xip_signed_0xC00.bin

  - # [Example of possible configuration #4]
    reset: true # [Required], Reset

```

In [19]:
SBX_CONFIG = WORKSPACE + "sbx_oem_closed.yaml"
SBX_FILE = WORKSPACE + "oem_closed.sbx"
OEM_SHARE_INPUT = WORKSPACE + "OEM_SHARE_INPUT.BIN"

%! nxpdevhsm $VERBOSITY generate -f $FAMILY -c $SBX_CONFIG -p $COM_PORT -w $SBX_WORKSPACE -i $OEM_SHARE_INPUT

# NOTE: By default device is reset after nxpdevhsm operation, if you don't want this behavior add --no-final-reset option

nxpdevhsm -v generate -f mwct2xd2 -c workspace/sbx_oem_closed.yaml -p com89 -w workspace/sbx_ws/ -i workspace/OEM_SHARE_INPUT.BIN 
INFO:spsdk.mboot.mcuboot:Connect: identifier='uart', device=com89
 1: Initial target reset is disabled 
 2: Generating OEM master share.
INFO:spsdk.mboot.mcuboot:CMD: WriteMemory(address=0x04003000, length=16, mem_id=0)
INFO:spsdk.mboot.mcuboot:CMD: GetProperty(MaxPacketSize, index=0)
INFO:spsdk.mboot.mcuboot:CMD: Status: 0 (0x0) Success.
INFO:spsdk.mboot.mcuboot:CMD: Max Packet Size = 32
INFO:spsdk.mboot.mcuboot:CMD: Status: 0 (0x0) Success.
INFO:spsdk.mboot.mcuboot:CMD: Successfully Send 16 out of 16 Bytes
INFO:spsdk.mboot.mcuboot:CMD: [TrustProvisioning] DSC HSM CREATE SESSION
INFO:spsdk.mboot.mcuboot:CMD: Status: 0 (0x0) Success.
INFO:spsdk.mboot.mcuboot:CMD: ReadMemory(address=0x04003100, length=52, mem_id=0)
INFO:spsdk.mboot.mcuboot:CMD: Status: 0 (0x0) Success.
INFO:spsdk.mboot.mcuboot:CMD: Successfully Received 52 from 52 Bytes
 3: Creating SBx head

In [20]:
SBX_FILE = WORKSPACE + "oem_closed.sbx"
# Final receive-sb-file
%! blhost -p $COM_PORT receive-sb-file $SBX_FILE

blhost -p com89 receive-sb-file workspace/oem_closed.sbx 
Sending SB file
Response status = 10119 (0x2787) ROM Loader: Pending Jump Command.
